# Stage 08 — Exploratory Data Analysis: NASDAQ Time Series

This notebook demonstrates comprehensive exploratory data analysis for NASDAQ historical data.

## Environment Setup

In [119]:
%%capture
%pip install -r requirements.txt

In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load Your NASDAQ Data

In [ ]:
df = pd.read_csv('../data/HistoricalData.csv')
df.rename(columns={ 'Close/Last': 'Close'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by='Date', ascending=True, inplace=True)
df.head(5)

,Date,Close,Open,High,Low
2516,2015-08-17,5091.70,5032.34,5092.69,5022.42
2515,2015-08-18,5059.35,5082.17,5085.14,5054.97
2514,2015-08-19,5019.05,5039.03,5060.93,4992.85
2513,2015-08-20,4877.49,4973.49,4986.51,4877.49
2512,2015-08-21,4706.04,4801.04,4856.75,4706.04


# Data Validation

In [122]:
df.isnull().sum()

Date     0
Close    0
Open     0
High     0
Low      0
dtype: int64

Verified the dataset for missing values \
The result shows **0 null values across all columns**, which means the dataset is complete and does not require any imputation or data cleaning for missing entries.  

In [11]:
df.describe()

,date,age,income,transactions,spend
count,160,160.000000,155.000000,160.00000,157.000000
mean,2021-04-21 12:00:00,40.018750,41983.866323,3.23750,117.291592
min,2021-02-01 00:00:00,22.000000,17928.800000,0.00000,0.540000
25%,2021-03-12 18:00:00,34.700000,32471.530000,2.00000,77.250000
50%,2021-04-21 12:00:00,40.150000,39332.520000,3.00000,119.320000
75%,2021-05-31 06:00:00,44.925000,49697.690000,4.00000,153.340000
max,2021-07-10 00:00:00,61.100000,87052.400000,20.00000,280.050000
std,NaN,8.458676,13262.457038,2.58561,51.768645


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          160 non-null    datetime64[ns]
 1   region        160 non-null    object        
 2   age           160 non-null    float64       
 3   income        155 non-null    float64       
 4   transactions  160 non-null    int64         
 5   spend         157 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 7.6+ KB
